<a href="https://colab.research.google.com/github/AlexandreGuerra-prod/Conversor-de-PDF/blob/main/Conversor_de_Faturas_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Conversor de Faturas de Cartão de Crédito (PDF) para Excel
# Compatível com Google Colab

# 🔹 Bloco 1: Instalar bibliotecas necessárias
!pip install pandas openpyxl tabula-py
# 🔹 Bloco 1: Instalar bibliotecas necessárias
!pip install pandas openpyxl tabula-py pymupdf regex

# %%


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 35.9 MB/s eta 0:00:00


In [6]:
# 🔹 Bloco 2: Importar bibliotecas e definir função principal
import pandas as pd
from google.colab import files
# Não usaremos tabula para este formato, mas a importação pode ficar
import tabula
import os
import fitz  # Importa PyMuPDF (fitz)
import re    # Importa a biblioteca de expressões regulares

# Verifica se Java está instalado (necessário para tabula, mas não para a nova abordagem de texto)
# Podemos remover ou manter, caso queira tentar tabula para outros PDFs
# os.system("apt-get install -y openjdk-11-jre > /dev/null 2>&1")
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

def fatura_pdf_para_excel(pdf_path, excel_path):
    print("📚 Lendo PDF como texto com PyMuPDF...")
    texto_completo = ""
    try:
        # Abre o documento PDF
        documento = fitz.open(pdf_path)
        # Extrai texto de cada página
        for numero_pagina in range(documento.page_count):
            pagina = documento.load_page(numero_pagina)
            texto_completo += pagina.get_text()
        documento.close()
        print("✅ Texto extraído do PDF.")
    except Exception as e:
        print(f"❌ Erro ao extrair texto do PDF: {e}")
        return # Sai da função em caso de erro na extração

    # --- NOVA ABORDAGEM: Processar texto com Regex ---
    print("🔍 Processando texto para encontrar lançamentos...")

    # Expressão regular para encontrar o padrão: Dia Mês Descrição Valor
    # Pode precisar de AJUSTES dependendo da variação no formato das linhas
    # Regex explicações:
    # ^\s*          -> Começa no início da linha, permitindo espaços iniciais opcionais
    # (\d{1,2})     -> Captura o dia (1 ou 2 dígitos), grupo 1
    # \s+           -> Um ou mais espaços
    # ([A-Za-z]+)   -> Captura o mês (uma ou mais letras), grupo 2
    # \s+           -> Um ou mais espaços
    # (.*?)         -> Captura a descrição (qualquer coisa), grupo 3, não ganancioso (para parar antes do valor)
    # \s+           -> Um ou mais espaços antes do valor
    # (R\$\s*\d{1,3}(?:\.\d{3})*(?:,\d{2})?) # Captura o valor (R$ opcional, números, ponto de milhar opcional, vírgula decimal opcional) - pode precisar de ajustes
    # Este regex para valor é mais complexo para tentar pegar formatos variados.
    # R\$?          -> R$ opcional
    # \s*           -> Espaços opcionais após R$
    # [\d.,]+       -> Pelo menos um dígito, ponto ou vírgula (captura a parte numérica do valor)

    # Tentativa 1 de Regex (mais simples, foca em Dia Mês e Valor no final)
    # ^\s*(\d{1,2})\s+([A-Za-z]+)\s+(.*?)\s+R\$?\s*([\d.,]+)$
    # Grupo 1: Dia (\d{1,2})
    # Grupo 2: Mês ([A-Za-z]+)
    # Grupo 3: Descrição (.*?)
    # Grupo 4: Valor ([\d.,]+)

    # Tentativa 2 de Regex (captura o R$ junto com o valor, se existir)
    # ^\s*(\d{1,2})\s+([A-Za-z]+)\s+(.*?)\s+(R\$?\s*[\d.,]+)$
    # Grupo 1: Dia
    # Grupo 2: Mês
    # Grupo 3: Descrição
    # Grupo 4: Valor (incluindo R$ opcional)

    # Vamos usar a tentativa 2 como base, mas pode precisar de ajustes finos
    # Dependendo se há outras coisas na linha ANTES da data ou DEPOIS do valor
    # Ou se o valor pode ser negativo, ter parênteses, etc.
    padrao_lancamento = re.compile(r'^\s*(\d{1,2})\s+([A-Za-z]+)\s+(.*?)\s+(R\$?\s*[\d.,]+)$', re.MULTILINE)


    dados_extraidos = []
    # Divide o texto em linhas para processar cada linha individualmente
    for linha in texto_completo.splitlines():
        match = padrao_lancamento.search(linha)
        if match:
            dia, mes, descricao, valor_str = match.groups()

            # Tenta padronizar a data (sem o ano, por enquanto)
            # Você precisará adicionar o ano da fatura manualmente ou tentar extraí-lo de outra parte do PDF
            data_str = f"{int(dia):02d}/{mes.upper()}" # Formato DD/MÊS (ex: 03/FEV)

            # Limpa e converte o valor para float
            # Remove R$, espaços, pontos de milhar (.), substitui vírgula (,) por ponto decimal (.)
            valor_limpo = valor_str.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            try:
                valor_float = float(valor_limpo)
            except ValueError:
                valor_float = None # Se não conseguir converter, define como None

            dados_extraidos.append({
                "Data_Texto": data_str, # Data no formato texto (DD/MÊS)
                "Descrição": descricao.strip(),
                "Valor_Bruto": valor_str.strip(), # Valor como foi encontrado no texto
                "Valor_Float": valor_float # Valor convertido para número
            })

    if not dados_extraidos:
        print("❌ Nenhum lançamento encontrado com o padrão definido. Verifique o formato das linhas ou ajuste o regex.")
        # Opcional: imprimir as primeiras linhas do texto para ajudar na depuração
        # print("\nPrimeiras 20 linhas do texto extraído para análise:")
        # print("\n".join(texto_completo.splitlines()[:20]))
        return # Sai da função se nada for encontrado

    print(f"✅ {len(dados_extraidos)} lançamentos encontrados.")

    # Cria um DataFrame com os dados extraídos
    df = pd.DataFrame(dados_extraidos)

    # --- Ajustes nas colunas e formatação final ---
    # Aqui você precisaria adicionar o ano para criar a coluna "Data" completa
    # Ex: Ano da fatura é 2023
    ano_fatura = "23" # <--- AJUSTE MANUALMENTE O ANO DA FATURA AQUI!
    try:
        df["Data"] = df["Data_Texto"] + "/" + ano_fatura
        # Converte para datetime para garantir formato DD/MM/YY
        df["Data"] = pd.to_datetime(df["Data"], format='%d/%b/%y', errors='coerce').dt.strftime('%d/%m/%y')
        # Remove linhas com data inválida após adicionar o ano
        df.dropna(subset=["Data"], inplace=True)
        print("✅ Coluna 'Data' criada e formatada.")
    except Exception as e:
         print(f"⚠️ Erro ao criar/formatar coluna 'Data' com ano: {e}")
         print("A coluna 'Data' pode estar faltando ou incompleta.")
         df["Data"] = df["Data_Texto"] # Se falhar, usa a data sem ano

    # Formatação final do valor para texto BR
    try:
        # Usa a coluna numérica se existir, senão tenta converter a bruta novamente
        if "Valor_Float" in df.columns:
             df.dropna(subset=["Valor_Float"], inplace=True) # Garante que temos números válidos
             df["Valor"] = df["Valor_Float"].map(lambda x: f"R${x:,.2f}".replace(",", "v").replace(".", ",").replace("v", "."))
        else:
             # Tenta converter a coluna bruta se a float falhou antes
             df["Valor"] = df["Valor_Bruto"].astype(str).str.replace(r'[R$\s.]', '', regex=True).str.replace(',', '.', regex=False)
             df["Valor"] = pd.to_numeric(df["Valor"], errors="coerce")
             df.dropna(subset=["Valor"], inplace=True)
             df["Valor"] = df["Valor"].map(lambda x: f"R${x:,.2f}".replace(",", "v").replace(".", ",").replace("v", "."))

        print("✅ Coluna 'Valor' formatada.")
    except Exception as e:
        print(f"⚠️ Erro ao formatar coluna 'Valor': {e}")
        print("A coluna 'Valor' pode estar com formato incorreto.")
        df["Valor"] = df["Valor_Bruto"] # Em caso de erro, usa o texto bruto


    # Seleciona e reordena as colunas finais
    colunas_finais = ["Data", "Descrição", "Valor"]
    # Verifica se as colunas finais existem antes de selecionar
    colunas_existentes = [col for col in colunas_finais if col in df.columns]
    df_final = df[colunas_existentes]

    if not colunas_existentes:
         print("❌ Nenhuma das colunas finais ('Data', 'Descrição', 'Valor') foi criada com sucesso.")
         # Opcional: Salvar o DataFrame bruto para inspeção
         df.to_excel("dados_brutos_extraidos.xlsx", index=False)
         print("💾 DataFrame bruto salvo como 'dados_brutos_extraidos.xlsx' para depuração.")
         return # Sai se não tiver as colunas essenciais

    print("💾 Salvando no Excel...")
    df_final.to_excel(excel_path, index=False)
    print(f"✅ Planilha gerada com sucesso: {excel_path}")

# %%


In [10]:
# 🔹 Bloco 3: Upload do PDF da fatura (SEM MUDANÇAS NESTE BLOCO)
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# %%

Saving Sicredi_Mar.pdf to Sicredi_Mar.pdf


In [11]:
# 🔹 Bloco 4: Converter e baixar o Excel (SEM MUDANÇAS NESTE BLOCO)
excel_path = "fatura_convertida.xlsx"
fatura_pdf_para_excel(pdf_path, excel_path)
files.download(excel_path)


📚 Lendo PDF como texto com PyMuPDF...
✅ Texto extraído do PDF.
🔍 Processando texto para encontrar lançamentos...
❌ Nenhum lançamento encontrado com o padrão definido. Verifique o formato das linhas ou ajuste o regex.


FileNotFoundError: Cannot find file: fatura_convertida.xlsx